In [1]:
import requests 

#download course FAQ documents
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Initiate Connection to Elasticsearch

In [2]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '66edb8a2bdfb', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'DI2vvLHuQ7ylsnyaRy5Qug', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Index the Data

In [3]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
try:
    # Check if index already exists
    if not es.indices.exists(index=index_name):
        # Create the index with the specified settings and mappings
        response = es.indices.create(index=index_name, body=index_settings)
        print(f"Index '{index_name}' created successfully.")
    else:
        print(f"Index '{index_name}' already exists.")
except Exception as e:
    print(f"Unexpected error: {e}")

Index 'course-questions' already exists.


In [4]:
#Index the documents
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Import minsearch

In [2]:
import sys
print("Python executable:", sys.executable)
!{sys.executable} -m pip list

Python executable: /home/ubuntu/.local/share/virtualenvs/llm-qa-system-PUouNnPP/bin/python
Package                   Version
------------------------- --------------
aiosqlite                 0.20.0
alembic                   1.13.2
annotated-types           0.7.0
anyio                     3.7.1
appdirs                   1.4.4
apprise                   1.8.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asgi-lifespan             2.1.0
asttokens                 2.4.1
async-lru                 2.0.4
async-timeout             4.0.3
asyncpg                   0.29.0
attrs                     23.2.0
Babel                     2.15.0
beautifulsoup4            4.12.3
bleach                    6.1.0
cachetools                5.3.3
certifi                   2024.6.2
cffi                      1.16.0
charset-normalizer        3.3.2
click                     8.1.7
cloudpickle               3.0.0
colorama                  0.4.6
comm                  

In [3]:
import minsearch